In [1]:
# 필수 라이브러리 설치 (첫 실행 시에만 필요)
!pip install transformers datasets torch torchvision evaluate kagglehub imbalanced-learn pillow pandas numpy matplotlib tqdm
!pip install "datasets>=2.16.1" "transformers>=4.38.2"

# 모든 필수 라이브러리 import
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import itertools
from collections import Counter
import matplotlib.pyplot as plt
import torch
import os
from pathlib import Path
from tqdm import tqdm

# PIL 및 이미지 처리 관련
from PIL import ImageFile

# Transformers 관련
from transformers import (
    TrainingArguments,
    Trainer,
    MobileViTImageProcessor,
    MobileViTV2ForImageClassification,
    MobileViTV2Config,
    DefaultDataCollator
)

# Dataset 관련
from datasets import Dataset, Image, ClassLabel

# 이미지 변환 관련
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomRotation,
    RandomResizedCrop,
    RandomHorizontalFlip,
    RandomAdjustSharpness,
    RandomAffine,
    ColorJitter,
    Resize,
    ToTensor
)

# 평가 및 샘플링 관련
import evaluate
from imblearn.over_sampling import RandomOverSampler

# 잘린 이미지 로드 허용
ImageFile.LOAD_TRUNCATED_IMAGES = True

print("모든 라이브러리가 성공적으로 import되었습니다.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
import kagglehub

# 최신 버전의 데이터셋 다운로드
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")
print("데이터셋 파일 경로:", path)

# 데이터 디렉토리 설정 (필요에 따라 수정)
data_dir = f"{path}/"
print(f"데이터 디렉토리: {data_dir}")

데이터셋 파일 경로: /kaggle/input/cifake-real-and-ai-generated-synthetic-images
데이터 디렉토리: /kaggle/input/cifake-real-and-ai-generated-synthetic-images/


In [3]:
# 레이블 설정
labels_list = ['REAL', 'FAKE']
label2id, id2label = dict(), dict()

# 고유 레이블들에 대해 ID 할당
for i, label in enumerate(labels_list):
    label2id[label] = i
    id2label[i] = label

ClassLabels = ClassLabel(num_classes=len(labels_list), names=labels_list)

# 레이블 매핑 함수
def map_label2id(example):
    example['label'] = ClassLabels.str2int(example['label'])
    return example

# 배치 데이터 준비를 위한 collate 함수
def collate_fn(examples):
    # 개별 예시들의 픽셀 값들을 하나의 텐서로 스택
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    # 레이블을 텐서로 변환
    labels = torch.tensor([example['label'] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

# 평가 메트릭 설정
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    # 모델 예측값 추출
    predictions = eval_pred.predictions
    # 실제 레이블 추출
    label_ids = eval_pred.label_ids

    # 가장 높은 확률을 가진 클래스 선택
    predicted_labels = predictions.argmax(axis=1)

    # 정확도 계산
    acc_score = accuracy.compute(predictions=predicted_labels, references=label_ids)['accuracy']

    return {
        "accuracy": acc_score
    }

print("유틸리티 함수들이 정의되었습니다.")


유틸리티 함수들이 정의되었습니다.


In [4]:
def get_vit_processor_and_transform(model="apple/mobilevitv2-1.0-imagenet1k-256"):
    processor = MobileViTImageProcessor.from_pretrained(model)

    # MobileViT 전용 정규화 파라미터
    normalize = Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])

    # 데이터 증강 강화
    _train_transforms = Compose([
        Resize((256, 256)),
        RandomRotation(30),
        RandomAffine(degrees=15, translate=(0.1, 0.1)),
        ColorJitter(brightness=0.2, contrast=0.2),
        ToTensor(),
        normalize
    ])

    # 검증 데이터용 변환 정의
    _val_transforms = Compose([
        Resize((256, 256)),  # ViT 모델 입력 크기로 리사이즈
        ToTensor(),           # 텐서로 변환
        normalize             # 정규화
    ])

    # 훈련 변환을 적용하는 함수
    def train_transforms(examples):
        examples['pixel_values'] = [_train_transforms(image.convert("RGB")) for image in examples['image']]
        return examples

    # 검증 변환을 적용하는 함수
    def val_transforms(examples):
        examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['image']]
        return examples

    return processor, train_transforms, val_transforms

def get_vit_model(model="apple/mobilevitv2-1.0-imagenet1k-256", num_labels=2):
    # 사전 훈련된 ViT 모델 로드
    config = MobileViTV2Config.from_pretrained(model)
    config.num_labels = num_labels
    config.id2label = id2label
    config.label2id = label2id

    # 모델 로드 시 옵션 추가
    model = MobileViTV2ForImageClassification.from_pretrained(
        model,
        config=config,
        ignore_mismatched_sizes=True  # 크기 불일치 무시
    )
    return model

print("ViT 모델 관련 함수들이 정의되었습니다.")


ViT 모델 관련 함수들이 정의되었습니다.


In [5]:
def get_dataset(data_dir: str = data_dir):
    # 파일명과 레이블을 저장할 빈 리스트 초기화
    file_names = []
    labels = []

    # 지정된 디렉토리의 모든 이미지 파일에 대해 반복
    for file in sorted((Path(data_dir).glob('*/*/*.*'))):
        label = str(file).split('/')[-2]  # 파일 경로에서 레이블 추출
        labels.append(label)  # 레이블을 리스트에 추가
        file_names.append(str(file))  # 파일 경로를 리스트에 추가

    # 수집된 파일명과 레이블로 pandas 데이터프레임 생성
    df = pd.DataFrame.from_dict({"image": file_names, "label": labels})

    # 소수 클래스의 랜덤 오버샘플링
    y = df[['label']]  # 타겟 변수 (레이블)
    df = df.drop(['label'], axis=1)  # 특성에서 레이블 분리

    # 랜덤 오버샘플러 객체 생성
    ros = RandomOverSampler(random_state=83)

    # 오버샘플링을 사용하여 데이터셋 리샘플링
    df, y_resampled = ros.fit_resample(df, y)

    # 리샘플링된 타겟 변수를 데이터프레임에 추가
    df['label'] = y_resampled

    # Hugging Face Dataset 객체로 변환
    dataset = Dataset.from_pandas(df).cast_column("image", Image())
    dataset = dataset.map(map_label2id, batched=True)

    # 레이블 컬럼을 ClassLabel 객체로 캐스팅
    dataset = dataset.cast_column('label', ClassLabels)

    # 훈련/테스트 분할 (60% 훈련, 40% 테스트)
    dataset = dataset.train_test_split(test_size=0.4, shuffle=True, stratify_by_column="label")

    train_data = dataset['train']
    test_data = dataset['test']

    return train_data, test_data

print("데이터 로더 함수가 정의되었습니다.")


데이터 로더 함수가 정의되었습니다.


In [6]:
# 데이터셋 로드
print("데이터셋을 로드하고 있습니다...")
train_data, test_data = get_dataset()

print(f"훈련 데이터 크기: {len(train_data)}")
print(f"테스트 데이터 크기: {len(test_data)}")

# 모델 및 전처리 설정
model_str = "apple/mobilevitv2-1.0-imagenet1k-256"
processor, train_transforms, val_transforms = get_vit_processor_and_transform(model_str)

# 데이터셋에 변환 적용
train_data.set_transform(train_transforms)
test_data.set_transform(val_transforms)

print("데이터 전처리가 완료되었습니다.")

# 데이터셋 샘플 확인
print("\n데이터셋 샘플:")
print(f"훈련 데이터 첫 번째 샘플 레이블: {train_data[0]['label']}")
print(f"테스트 데이터 첫 번째 샘플 레이블: {test_data[0]['label']}")


데이터셋을 로드하고 있습니다...


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/120000 [00:00<?, ? examples/s]

훈련 데이터 크기: 72000
테스트 데이터 크기: 48000


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

데이터 전처리가 완료되었습니다.

데이터셋 샘플:
훈련 데이터 첫 번째 샘플 레이블: 0
테스트 데이터 첫 번째 샘플 레이블: 1


In [ ]:
# 모델 로드
print("ViT 모델을 로드하고 있습니다...")
model = get_vit_model(model_str)

# 훈련 매개변수 설정
model_name = "ai_vs_real_image_detection"
num_train_epochs = 10

args = TrainingArguments(
    output_dir=model_name,
    logging_dir='./logs',
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=num_train_epochs,
    weight_decay=0.02,
    warmup_steps=50,
    remove_unused_columns=False,
    save_strategy='epoch',
    load_best_model_at_end=True,
    save_total_limit=1,
    report_to="none"
)

print("훈련 매개변수가 설정되었습니다.")
print(f"에포크 수: {num_train_epochs}")
print(f"학습률: {args.learning_rate}")
print(f"배치 크기: {args.per_device_train_batch_size}")


ViT 모델을 로드하고 있습니다...


config.json:   0%|          | 0.00/69.8k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/19.8M [00:00<?, ?B/s]

Some weights of MobileViTV2ForImageClassification were not initialized from the model checkpoint at apple/mobilevitv2-1.0-imagenet1k-256 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 512]) in the checkpoint and torch.Size([2, 512]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


훈련 매개변수가 설정되었습니다.
에포크 수: 20
학습률: 3e-05
배치 크기: 32


In [8]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")

Total parameters: 4389867
Trainable parameters: 4389867


In [ ]:
# Trainer 설정
trainer = Trainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

print("Trainer가 설정되었습니다.")

# 훈련 전 초기 평가
print("\n=== 훈련 전 초기 평가 ===")
initial_eval = trainer.evaluate()
print(f"초기 정확도: {initial_eval['eval_accuracy']:.4f}")


Trainer가 설정되었습니다.

=== 훈련 전 초기 평가 ===


model.safetensors:   0%|          | 0.00/19.7M [00:00<?, ?B/s]

초기 정확도: 0.5027


In [ ]:
# 모델 훈련 시작
print("\n=== 모델 훈련 시작 ===")
trainer.train()


=== 모델 훈련 시작 ===


Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy
1,0.188500,0.194255,0.019700,0.925646
2,0.151800,0.181458,0.019700,0.927229
3,0.140900,0.130642,0.019700,0.948937
4,0.125900,0.113559,0.019700,0.956333
5,0.114200,0.113068,0.019700,0.958042
6,0.109300,0.096179,0.019700,0.964167
7,0.102100,0.094064,0.019700,0.965083
8,0.089400,0.103587,0.019700,0.962688
9,0.092200,0.123518,0.019700,0.957729
10,0.083300,0.084893,0.019700,0.968875


In [ ]:
# 훈련 후 최종 평가
print("\n=== 훈련 후 최종 평가 ===")
final_eval = trainer.evaluate()
print(f"최종 정확도: {final_eval['eval_accuracy']:.4f}")